In [1]:
import json
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.stats import linregress

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


## Extract data

In [2]:
files = glob.glob(r'C:\Users\trist\OneDrive\Documents\game_data\valorant_raw/*.json')
len(files)

2782

In [3]:
with open(files[0], 'r') as f:
    json_data = json.load(f)
json_data = json.loads(json_data)
json_data['data'].keys()

dict_keys(['attributes', 'metadata', 'segments', 'expiryDate'])

In [4]:
files[0]

'C:\\Users\\trist\\OneDrive\\Documents\\game_data\\valorant_raw\\0001be30-1ef6-40d8-8f92-13ffbd093cf1.json'

In [5]:
json_data['data']['attributes']

{'id': '0001be30-1ef6-40d8-8f92-13ffbd093cf1'}

In [6]:
json_data['data']['metadata']

{'modeKey': 'bomb',
 'modeName': 'Normal',
 'modeImageUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/icons/modes/normal.png',
 'modeMaxRounds': 25,
 'duration': 2664924,
 'dateStarted': '2022-07-08T06:53:09.588+00:00',
 'rounds': 24,
 'isRanked': False,
 'queueId': 'unrated',
 'map': '2fb9a4fd-47b8-4e7d-a969-74b4046ebd53',
 'mapName': 'Breeze',
 'mapImageUrl': 'https://titles.trackercdn.com/valorant-api/maps/2fb9a4fd-47b8-4e7d-a969-74b4046ebd53/splash.png'}

In [7]:
game_metadata = json_data['data']['metadata']

In [8]:
json_data['data']['expiryDate']

'2022-08-08T18:11:36.1374295+00:00'

In [9]:
all_segments_types = set()

for i in json_data['data']['segments']:
    all_segments_types.add(i['type'])

all_segments_types

{'player-round',
 'player-round-damage',
 'player-round-kills',
 'player-summary',
 'round-summary',
 'team-summary'}

In [10]:
player_rounds = [i for i in json_data['data']['segments'] if i['type'] == 'player-round']
player_rounds_damage = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-damage']
player_rounds_kills = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-kills']
player_summary = [i for i in json_data['data']['segments'] if i['type'] == 'player-summary']
round_summary = [i for i in json_data['data']['segments'] if i['type'] == 'round-summary']
team_summary = [i for i in json_data['data']['segments'] if i['type'] == 'team-summary']

## player_rounds_kills

In [11]:
player_rounds_kills[0].keys()

dict_keys(['type', 'attributes', 'metadata', 'expiryDate', 'stats'])

In [12]:



player_rounds_kills[0]['attributes']

{'round': 1,
 'platformSlug': 'riot',
 'platformUserIdentifier': 'rimo#SEXY',
 'opponentPlatformSlug': 'riot',
 'opponentPlatformUserIdentifier': 'Hennessyy#AKA47'}

In [13]:
player_rounds_kills[0]['metadata']

{'platformInfo': {'platformSlug': 'riot',
  'platformUserId': None,
  'platformUserHandle': 'rimo#SEXY',
  'platformUserIdentifier': 'rimo#SEXY',
  'avatarUrl': None,
  'additionalParameters': None},
 'opponentPlatformInfo': {'platformSlug': 'riot',
  'platformUserId': None,
  'platformUserHandle': 'Hennessyy#AKA47',
  'platformUserIdentifier': 'Hennessyy#AKA47',
  'avatarUrl': None,
  'additionalParameters': None},
 'opponentLocation': {'x': 4917, 'y': 3223},
 'playerLocations': [{'puuid': 'gaxPhPH2L0l07DbnPvzTzKcQzNi3qJfuNjigpnUV3UBkRNdDpGdhyhbU2KWVV1QbHIAXZrowoPOkcA',
   'viewRadians': 2.4492486,
   'location': {'x': 5927, 'y': 4912}},
  {'puuid': '-2GqRI2DorSneRUZgv2T47wOha66l9v3qUbG7Pxw1VibAdJkrHnLI6WgCZeAj3O8HUtAuAyuD4suLQ',
   'viewRadians': 5.6062036,
   'location': {'x': 4806, 'y': 5496}},
  {'puuid': 'LyIVoyMdyMEmaLGGwwI7eyO0vc6UIlwreIZF7OaidgY0SyLa4vF3Q6jGLlB_KBtCkhHfMtTNYaWi1A',
   'viewRadians': 5.0044217,
   'location': {'x': 3969, 'y': 6342}},
  {'puuid': 'q8cbMuUjAnP0Hj

In [14]:
player_rounds_kills[0]['stats']

{'damage': {'rank': None,
  'percentile': None,
  'displayName': None,
  'displayCategory': None,
  'category': None,
  'metadata': {},
  'value': 100,
  'displayValue': '100',
  'displayType': 'Number'}}

In [15]:
files = glob.glob(r'C:\Users\trist\OneDrive\Documents\game_data\valorant_raw/*.json')
print(len(files))
kill_records = list()

for file in files:
    with open(file, 'r') as f:
        json_data = json.load(f)
    json_data = json.loads(json_data)
    player_rounds_kills = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-kills']
    for kill in player_rounds_kills:        
        kill_records.append({'weaponName':kill['metadata']['weaponName'],
                            'round':kill['attributes']['round'],
                            'roundTime':kill['metadata']['roundTime']})
kill_df = pd.DataFrame.from_dict(kill_records)
kill_df.head()

2782


,weaponName,round,roundTime
0,Classic,1,20725
1,Ghost,1,37182
2,Classic,1,51285
3,Classic,1,23532
4,Classic,1,27015


In [16]:
kill_df['weaponName'].value_counts().index.tolist()

['Vandal',
 'Phantom',
 'Spectre',
 'Ghost',
 'Classic',
 'Operator',
 'Sheriff',
 'Guardian',
 'Marshal',
 'Odin',
 'Judge',
 'Bulldog',
 'Stinger',
 'Frenzy',
 'Ares',
 'Shorty',
 'Bucky']

## round_summary

In [17]:
round_summary[19]

{'type': 'round-summary',
 'attributes': {'round': 20},
 'metadata': {},
 'expiryDate': '0001-01-01T00:00:00+00:00',
 'stats': {'roundResult': {'rank': None,
   'percentile': None,
   'displayName': None,
   'displayCategory': None,
   'category': None,
   'metadata': {},
   'value': 'Detonate',
   'displayValue': 'Detonate',
   'displayType': 'String'},
  'winningTeam': {'rank': None,
   'percentile': None,
   'displayName': None,
   'displayCategory': None,
   'category': None,
   'metadata': {},
   'value': 'Blue',
   'displayValue': 'Blue',
   'displayType': 'String'}}}

## player_rounds

In [18]:
player_rounds[17]

{'type': 'player-round',
 'attributes': {'round': 2,
  'platformSlug': 'riot',
  'platformUserIdentifier': 'Corgi Tree#NA1'},
 'metadata': {'teamId': 'Red',
  'agentKey': '707eab51-4836-f488-046a-cda6bf494859',
  'platformInfo': {'platformSlug': 'riot',
   'platformUserId': None,
   'platformUserHandle': 'Corgi Tree#NA1',
   'platformUserIdentifier': 'Corgi Tree#NA1',
   'avatarUrl': None,
   'additionalParameters': None},
  'hasWon': False},
 'expiryDate': '0001-01-01T00:00:00+00:00',
 'stats': {'score': {'rank': None,
   'percentile': None,
   'displayName': 'Score',
   'displayCategory': None,
   'category': None,
   'metadata': {},
   'value': 300,
   'displayValue': '300',
   'displayType': 'Number'},
  'kills': {'rank': None,
   'percentile': None,
   'displayName': 'Kills',
   'displayCategory': None,
   'category': None,
   'metadata': {},
   'value': 1,
   'displayValue': '1',
   'displayType': 'Number'},
  'deaths': {'rank': None,
   'percentile': None,
   'displayName': 'Dea

## round_summary

In [19]:
len(round_summary)

24

In [20]:
# round_summary[22]


In [21]:
def get_winner(round_summary):
    return round_summary[-1]['stats']['winningTeam']['value']

get_winner(round_summary)

'Red'

## player_summary

In [22]:
len(player_summary)

10

In [23]:
player_summary[0].keys()

dict_keys(['type', 'attributes', 'metadata', 'expiryDate', 'stats'])

In [24]:
player_summary[0]['attributes']

{'platformSlug': 'riot', 'platformUserIdentifier': 'Hennessyy#AKA47'}

In [25]:
player_summary[0]['metadata']

{'partyId': '4155c5e2-bf8d-4736-acb1-8ec95e2576b1',
 'teamId': 'Blue',
 'agentKey': 'add6443a-41bd-e414-f6ad-e58d267f4e95',
 'agentName': 'Jett',
 'agentColor': '#98DCFF',
 'agentImageUrl': 'https://titles.trackercdn.com/valorant-api/agents/add6443a-41bd-e414-f6ad-e58d267f4e95/displayicon.png',
 'agentPortraitUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/db/agents/jett_portrait.png',
 'countryCode': None,
 'platformInfo': {'platformSlug': 'riot',
  'platformUserId': None,
  'platformUserHandle': 'Hennessyy#AKA47',
  'platformUserIdentifier': 'Hennessyy#AKA47',
  'avatarUrl': None,
  'additionalParameters': None}}

In [26]:
player_summary[0]['stats'].keys()

dict_keys(['rank', 'currRank', 'score', 'scorePerRound', 'killsPerRound', 'kills', 'deaths', 'assists', 'kdRatio', 'damage', 'damagePerRound', 'singleKills', 'doubleKills', 'tripleKills', 'quadraKills', 'pentaKills', 'multiKills', 'grenadeCasts', 'ability1Casts', 'ability2Casts', 'ultimateCasts', 'grenadeCastsPerRound', 'ability1CastsPerRound', 'ability2CastsPerRound', 'ultimateCastsPerRound', 'plants', 'defuses', 'firstKills', 'firstDeaths', 'esr', 'firstKillsPerRound', 'firstDeathsPerRound', 'econRating', 'hsAccuracy', 'kast'])

In [27]:
player_summary[0]['stats']['rank']

{'rank': None,
 'percentile': None,
 'displayName': 'Rating',
 'displayCategory': None,
 'category': 'mmr',
 'metadata': {'iconUrl': 'https://trackercdn.com/cdn/tracker.gg/valorant/icons/tiersv2/0.png'},
 'value': 'Unranked',
 'displayValue': 'Unranked',
 'displayType': 'String'}

In [28]:
player_summary[0]['stats']['score']

{'rank': None,
 'percentile': None,
 'displayName': 'Score',
 'displayCategory': None,
 'category': None,
 'metadata': {},
 'value': 7621,
 'displayValue': '7,621',
 'displayType': 'Number'}

In [29]:
player_summary[0]['stats']['scorePerRound']

{'rank': None,
 'percentile': None,
 'displayName': 'Score per Round',
 'displayCategory': None,
 'category': None,
 'metadata': {},
 'value': 317.5416666666667,
 'displayValue': '318',
 'displayType': 'Number'}

## Win prediction data set creation

In [30]:
all_agents_list = ['Astra',
         'Breach',
         'Brimstone',
         'Chamber',
         'Cypher',
         'Fade',
         'Jett',
         'KAY/O',
         'Killjoy',
         'Neon',
         'Omen',
         'Phoenix',
         'Raze',
         'Reyna',
         'Sage',
         'Skye',
         'Sova',
         'Viper',
         'Yoru']

agent_roles = {'Astra':'Controller',
              'Breach':'Initiator',
         'Brimstone':'Controller',
         'Chamber':'Sentinel',
         'Cypher':'Sentinel',
         'Fade':'Initiator',
         'Jett':'Duelist',
         'KAY/O':'Initiator',
         'Killjoy':'Sentinel',
         'Neon':'Duelist',
         'Omen':'Controller',
         'Phoenix':'Duelist',
         'Raze':'Duelist',
         'Reyna':'Duelist',
         'Sage':'Sentinel',
         'Skye':'Initiator',
         'Sova':'Initiator',
         'Viper':'Controller',
         'Yoru':'Duelist'}
roles_list = ['Controller', 'Initiator', 'Duelist', 'Sentinel']
all_maps_list = ['Ascent', 
                 'Bind', 
                 'Breeze', 
                 'Fracture', 
                 'Haven', 
                 'Icebox', 
                 'Pearl', 
                 'Split']
weapons = ['Vandal',
             'Phantom',
             'Spectre',
             'Ghost',
             'Classic',
             'Operator',
             'Sheriff',
             'Guardian',
             'Marshal',
             'Odin',
             'Judge',
             'Bulldog',
             'Stinger',
             'Frenzy',
             'Ares',
             'Shorty',
             'Bucky']

In [31]:
invalid_keys = [
    'currRank',
]

def extract_player_rows(game_metadata, round_summary, player_summary, player_rounds_kills):
    winning_team = get_winner(round_summary)
    game_datetime = game_metadata['dateStarted']
    map_pick = game_metadata['mapName']
    
    data = list()
    
    for i in player_summary:
        new_record = dict()
        new_record['name'] = i['attributes']['platformUserIdentifier']
        
        agent = i['metadata']['agentName']
        agent_role = agent_roles[agent]
        
        new_record['game_datetime'] = game_datetime
        new_record['won_game'] = int(winning_team == i['metadata']['teamId'])
        new_record['team'] = i['metadata']['teamId']
        new_record['map_pick'] = map_pick
        
        for j in i['stats'].keys():
            if j in invalid_keys:
                continue
            new_record[j] = i['stats'][j]['value']
        
        for j in roles_list:
            if j == agent_role:
                new_record[f'role_{j}'] = 1
            else:
                new_record[f'role_{j}'] = 0
        
        for j in all_agents_list:
            if j == agent:
                new_record[f'agent_{j}'] = 1
            else:
                new_record[f'agent_{j}'] = 0
        
        weapon_kills_dict = {j: 0 for j in weapons}
        for j in player_rounds_kills:
            if 'platformInfo' in j and i['attributes']['platformUserIdentifier'] != j['platformInfo']['platformUserHandle']:
                if j['metadata']['weaponName'] in weapons:
                    weapon_kills_dict[j['metadata']['weaponName']] += 1
            
        data.append(new_record)
            
    return data
        
        
# processed_match_records = extract_player_rows(game_metadata, round_summary, player_summary, player_rounds_kills)
# processed_match_records[8]

In [32]:


def create_agent_row(agent_list, map_pick, game_win):

    
    for i in agent_list:
        if i not in all_agents_list:
            raise Exception(f'Invalid agent: {i}')
            
    if map_pick not in all_maps_list:
        raise Exception(f'Invalid map: {map_pick}')
    
    row = dict()
    
    roles_dict = {f'role_{i}': 0 for i in roles_list}
    
    for i in all_agents_list:
        row['agent_' + i] = int(i in agent_list)
                
    for i in agent_list:
        roles_dict[f'role_{agent_roles[i]}'] += 1
    
    roles_dict_keys = list(roles_dict.keys())
    
    for i in roles_dict_keys:
        for j in roles_dict_keys:
            if roles_list.index(i.split('_')[-1]) >= roles_list.index(j.split('_')[-1]):
                continue
            roles_dict[f'{i}*{j}'] = roles_dict[i] * roles_dict[j]
        
    for i in all_maps_list:
        row['map_' + i] = int(i == map_pick)
        
    row.update(roles_dict)
        
    row['game_win'] = game_win
    return row


def extract_team_rows(game_metadata, game_attributes, round_summary, player_summary):
    
    team_red_agents = list()
    team_blue_agents = list()
    
    map_pick = game_metadata['mapName']
        
    winning_team = get_winner(round_summary)
    
    for i in player_summary:
        if i['metadata']['teamId'] == 'Red':
            team_red_agents.append(i['metadata']['agentName'])
        if i['metadata']['teamId'] == 'Blue':
            team_blue_agents.append(i['metadata']['agentName'])
            
    row1 = create_agent_row(team_red_agents, map_pick, int('Red' == winning_team))
    row2 = create_agent_row(team_blue_agents, map_pick, int('Blue' == winning_team))
    
    return [row1, row2]
    
    




In [33]:
def get_all_processed_data(): 
    files = glob.glob(r'C:\Users\trist\OneDrive\Documents\game_data\valorant_raw/*.json')
    
    all_records = list()
    agent_records = list()
    
    for file in files:
        
        with open(file, 'r') as f:
            json_data = json.load(f)
        json_data = json.loads(json_data)
        if 'data' not in json_data:
            print(f'error {file}')
            continue
        game_metadata = json_data['data']['metadata']
        game_attributes = json_data['data']['attributes']

        player_rounds = [i for i in json_data['data']['segments'] if i['type'] == 'player-round']
        player_rounds_damage = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-damage']
        player_rounds_kills = [i for i in json_data['data']['segments'] if i['type'] == 'player-round-kills']
        player_summary = [i for i in json_data['data']['segments'] if i['type'] == 'player-summary']
        round_summary = [i for i in json_data['data']['segments'] if i['type'] == 'round-summary']
        team_summary = [i for i in json_data['data']['segments'] if i['type'] == 'team-summary']
        
        all_records.extend(extract_player_rows(game_metadata, round_summary, player_summary, player_rounds_kills))
        agent_records.extend(extract_team_rows(game_metadata, game_attributes, round_summary, player_summary))
    return all_records, agent_records
        
all_records, all_agent_records = get_all_processed_data()
len(all_records), len(all_agent_records)


(27920, 5576)

In [34]:
user_record_count = dict()

for i in all_records:
    user_record_count.setdefault(i['name'], 0)
    user_record_count[i['name']] += 1

user_record_count_sorted = list()

for k, v in user_record_count.items():
    user_record_count_sorted.append({'user':k, 'count':v})

sorted(user_record_count_sorted, key = lambda x: x['count'], reverse = True)[:20]


[{'user': 'strawberry milk#우유갠디', 'count': 37},
 {'user': 'Thug Twerker#money', 'count': 30},
 {'user': 'CV WorstNub#2822', 'count': 28},
 {'user': 'tenz jr#333', 'count': 28},
 {'user': 'SmokedSalmon#4492', 'count': 28},
 {'user': 'IAmTempest#NA1', 'count': 26},
 {'user': 'Daddy Brim#OTP', 'count': 26},
 {'user': 'sussy#queer', 'count': 26},
 {'user': 'BishWhoDontMiss#6558', 'count': 26},
 {'user': '32Equr#6990', 'count': 25},
 {'user': 'YungCheddaBigD1k#2594', 'count': 25},
 {'user': 'Barbie#nails', 'count': 25},
 {'user': 'ZenitsuAG#7485', 'count': 24},
 {'user': 'I bark for mommy#6969', 'count': 24},
 {'user': 'CandyMan#7591', 'count': 23},
 {'user': 'Tan#0008', 'count': 23},
 {'user': 'DyslexicNepali#4122', 'count': 23},
 {'user': 'Arowana#queer', 'count': 23},
 {'user': 'DaddyMingze#NA1', 'count': 22},
 {'user': 'wakefull#icy', 'count': 22}]

In [35]:
all_records = [i for i in all_records if user_record_count[i['name']] >= 5]
len(all_records)

5287

In [36]:
all_records_df = pd.DataFrame.from_dict(all_records)
all_records_df

,name,game_datetime,won_game,team,map_pick,rank,score,scorePerRound,killsPerRound,kills,...,agent_Neon,agent_Omen,agent_Phoenix,agent_Raze,agent_Reyna,agent_Sage,agent_Skye,agent_Sova,agent_Viper,agent_Yoru
0,Yor#rice,2022-07-08T06:53:09.588+00:00,1,Red,Breeze,Unranked,5623,234.291667,0.833333,20,...,0,0,0,0,0,0,1,0,0,0
1,hardstuckiron#3570,2022-08-07T22:08:58.634+00:00,1,Blue,Icebox,Bronze 1,6042,402.800000,1.466667,22,...,0,0,0,0,0,0,0,0,0,0
2,Andreww#1178,2022-08-07T22:08:58.634+00:00,1,Blue,Icebox,Iron 3,2032,135.466667,0.466667,7,...,0,0,1,0,0,0,0,0,0,0
3,Boldzero#101,2022-07-27T05:21:43.67+00:00,1,Red,Ascent,Bronze 3,1052,210.400000,0.800000,4,...,0,1,0,0,0,0,0,0,0,0
4,DarkPegasis123#NA1,2022-08-06T17:49:51.751+00:00,0,Blue,Pearl,Gold 3,2898,144.900000,0.400000,8,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5282,Hitchman224#1988,2022-07-30T03:04:58.767+00:00,1,Red,Ascent,Unranked,3455,215.937500,0.750000,12,...,0,0,0,0,0,0,0,0,0,0
5283,frost2094#4441,2022-08-06T20:27:11.453+00:00,1,Red,Icebox,Bronze 3,472,78.666667,0.333333,2,...,0,0,0,0,0,0,0,0,0,0
5284,SmokedSalmon#4492,2022-07-27T04:07:20.866+00:00,1,Blue,Haven,Gold 2,4892,188.153846,0.615385,16,...,0,0,0,1,0,0,0,0,0,0
5285,Mrpancake#31798,2022-07-27T04:07:20.866+00:00,1,Blue,Haven,Silver 2,4089,157.269231,0.538462,14,...,0,0,0,0,0,0,0,0,0,0


In [37]:
all_records_df['name'].nunique()

373

In [38]:
all_records_df['game_datetime'].min()

'2022-03-26T16:46:26.008+00:00'

In [39]:
all_records_df['map_pick'].value_counts(normalize=True)

Fracture    0.149234
Haven       0.146964
Ascent      0.146775
Breeze      0.140912
Bind        0.139399
Pearl       0.136372
Icebox      0.133724
Split       0.006620
Name: map_pick, dtype: float64

In [40]:
all_records_df['killsPerRound'].describe()

count    5287.000000
mean        0.791541
std         1.305405
min         0.000000
25%         0.500000
50%         0.681818
75%         0.904762
max        39.000000
Name: killsPerRound, dtype: float64

In [41]:
all_records_df['rank'].value_counts(normalize=True)

Bronze 1      0.123510
Bronze 2      0.121619
Bronze 3      0.098165
Silver 1      0.091356
Silver 3      0.089843
Unranked      0.080575
Iron 3        0.072064
Silver 2      0.062228
Gold 1        0.056743
Gold 2        0.052582
Gold 3        0.045205
Iron 2        0.038207
Platinum 1    0.026480
Platinum 3    0.015510
Platinum 2    0.013240
Diamond 2     0.005674
Iron 1        0.005485
Diamond 1     0.001513
Name: rank, dtype: float64

In [42]:

def get_player_features(player_df):
    player_df = player_df.sort_values(by = ['game_datetime'])

    data = list()


    row_counter = 0
    for n, (idx, row) in enumerate(player_df.iterrows()):
        if n < 5:
            continue

        past_game = player_df.iloc[n-1:n].mean().to_dict()
        past_5_games = player_df.iloc[n-5:n].mean().to_dict()

        new_x = dict()

#         for k, v in past_game.items():
#             new_x[f'past_game_{k}'] = v

        for k, v in past_5_games.items():
            new_x[f'past_5_games_avg_{k}'] = v


        new_x['future_won_game'] =  row['won_game']
        data.append(new_x)


    data_df = pd.DataFrame.from_dict(data)
    data_df = data_df.fillna(0)
    return data_df



In [43]:

training_players, val_players = train_test_split(list(set(all_records_df['name'].tolist())))

training_data_dfs = list()
val_data_dfs = list()
all_data_dfs= list()

for i in training_players:
    
    player_df = all_records_df[all_records_df['name'] == i]
    player_data_df = get_player_features(player_df)
    training_data_dfs.append(player_data_df)

    
for i in val_players:
    
    player_df = all_records_df[all_records_df['name'] == i]
    player_data_df = get_player_features(player_df)
    val_data_dfs.append(player_data_df)
    
for i in list(set(all_records_df['name'].tolist())):
    
    player_df = all_records_df[all_records_df['name'] == i]
    player_data_df = get_player_features(player_df)
    all_data_dfs.append(player_data_df)


    
training_data_df = pd.concat(training_data_dfs)
val_data_df = pd.concat(val_data_dfs)
all_data_df = pd.concat(all_data_dfs)

len(training_players), len(val_players)

(279, 94)

In [44]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# training_data_df.corr().sort_values(by=['future_won_game'])[['future_won_game']]

In [45]:
# training_data_df.isna().sum()

In [46]:
rf = RandomForestClassifier()

x = training_data_df.drop('future_won_game', axis = 1)
y = training_data_df['future_won_game']

x_val = val_data_df.drop('future_won_game', axis = 1)
y_val = val_data_df['future_won_game']

rf.fit(x, y)

preds = rf.predict(x_val)

accuracy_score(preds, y_val)

0.5349127182044888

In [47]:
x

,past_5_games_avg_won_game,past_5_games_avg_score,past_5_games_avg_scorePerRound,past_5_games_avg_killsPerRound,past_5_games_avg_kills,past_5_games_avg_deaths,past_5_games_avg_assists,past_5_games_avg_kdRatio,past_5_games_avg_damage,past_5_games_avg_damagePerRound,past_5_games_avg_singleKills,past_5_games_avg_doubleKills,past_5_games_avg_tripleKills,past_5_games_avg_quadraKills,past_5_games_avg_pentaKills,past_5_games_avg_multiKills,past_5_games_avg_grenadeCasts,past_5_games_avg_ability1Casts,past_5_games_avg_ability2Casts,past_5_games_avg_ultimateCasts,past_5_games_avg_grenadeCastsPerRound,past_5_games_avg_ability1CastsPerRound,past_5_games_avg_ability2CastsPerRound,past_5_games_avg_ultimateCastsPerRound,past_5_games_avg_plants,past_5_games_avg_defuses,past_5_games_avg_firstKills,past_5_games_avg_firstDeaths,past_5_games_avg_esr,past_5_games_avg_firstKillsPerRound,past_5_games_avg_firstDeathsPerRound,past_5_games_avg_econRating,past_5_games_avg_hsAccuracy,past_5_games_avg_kast,past_5_games_avg_role_Controller,past_5_games_avg_role_Initiator,past_5_games_avg_role_Duelist,past_5_games_avg_role_Sentinel,past_5_games_avg_agent_Astra,past_5_games_avg_agent_Breach,past_5_games_avg_agent_Brimstone,past_5_games_avg_agent_Chamber,past_5_games_avg_agent_Cypher,past_5_games_avg_agent_Fade,past_5_games_avg_agent_Jett,past_5_games_avg_agent_KAY/O,past_5_games_avg_agent_Killjoy,past_5_games_avg_agent_Neon,past_5_games_avg_agent_Omen,past_5_games_avg_agent_Phoenix,past_5_games_avg_agent_Raze,past_5_games_avg_agent_Reyna,past_5_games_avg_agent_Sage,past_5_games_avg_agent_Skye,past_5_games_avg_agent_Sova,past_5_games_avg_agent_Viper,past_5_games_avg_agent_Yoru
0,0.6,3381.4,172.564392,0.576605,11.4,12.8,5.8,0.839427,2041.6,103.134183,4.0,2.2,1.0,0.0,0.0,1.0,12.2,12.6,22.8,2.2,0.667688,0.645284,1.212812,0.102716,4.8,4.8,1.0,1.2,63.333333,0.047605,0.069410,48.4,13.923515,84.802768,0.6,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0
1,0.6,3976.2,186.077487,0.644462,13.8,15.0,5.4,0.915897,2441.8,114.375850,5.4,2.0,1.2,0.0,0.2,1.4,13.0,12.2,29.0,2.4,0.612926,0.564332,1.353288,0.112240,5.8,5.8,1.2,1.2,73.333333,0.057129,0.053934,51.8,14.849441,64.326577,0.8,0.2,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0
2,0.4,3533.8,174.240698,0.594963,12.2,14.4,4.0,0.819231,2185.0,108.635382,4.4,2.0,1.0,0.0,0.2,1.2,12.2,11.4,24.2,2.0,0.665100,0.603128,1.184726,0.101537,4.4,4.4,1.0,1.2,63.333333,0.048434,0.060623,48.2,14.296250,64.259688,1.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0
3,0.4,3498.0,160.516575,0.544963,12.0,15.2,3.6,0.738688,2210.0,102.567399,5.0,1.8,0.6,0.2,0.2,1.0,12.0,10.2,30.4,1.8,0.610714,0.518040,1.410165,0.084432,3.6,3.6,0.8,1.8,28.333333,0.035714,0.085623,44.2,13.695157,62.417582,1.0,0.0,0.0,0.0,0.2,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.0
4,0.2,3717.0,173.969708,0.623660,13.4,15.4,3.2,0.833193,2301.8,108.899730,6.4,1.8,0.6,0.2,0.2,1.0,14.4,10.6,25.4,2.2,0.743045,0.550120,1.191117,0.106487,2.8,2.8,0.8,1.4,41.666667,0.036717,0.066575,46.4,16.259259,64.773472,1.0,0.0,0.0,0.0,0.2,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,0.6,4586.6,230.668254,0.763968,15.2,14.2,6.0,1.173452,3157.2,158.929683,6.0,2.6,1.4,0.0,0.0,1.4,15.6,6.8,20.6,2.6,0.785397,0.337778,1.044286,0.130794,2.0,2.0,3.2,2.4,40.000000,0.157937,0.123016,57.6,28.308724,66.714286,0.2,0.2,0.4,0.2,0.0,0.0,0.0,0.2,0.0,0.2,0.4,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.6,4993.6,257.008254,0.886190,17.2,14.0,5.0,1.370422,3318.2,170.750794,6.6,3.0,1.6,0.0,0.0,1.6,17.6,8.0,22.0,2.8,0.902063,0.410000,1.138730,0.144127,2.6,2.6,4.4,2.2,50.555556,0.225714,0.115238,50.8,30.435708

In [48]:
fi_list = list()
for i, j in zip(x_val.columns, rf.feature_importances_):
    fi_list.append({'column':i, 'importance':j})
    
pd.DataFrame.from_dict(fi_list).sort_values(by = ['importance'])[-10:]

,column,importance
7,past_5_games_avg_kdRatio,0.031752
16,past_5_games_avg_grenadeCasts,0.031971
8,past_5_games_avg_damage,0.032054
9,past_5_games_avg_damagePerRound,0.032368
28,past_5_games_avg_esr,0.032531
20,past_5_games_avg_grenadeCastsPerRound,0.032647
22,past_5_games_avg_ability2CastsPerRound,0.033371
32,past_5_games_avg_hsAccuracy,0.033911
31,past_5_games_avg_econRating,0.033961
33,past_5_games_avg_kast,0.035281


In [49]:
rf = LogisticRegression()

x = training_data_df.drop('future_won_game', axis = 1)
y = training_data_df['future_won_game']

x_val = val_data_df.drop('future_won_game', axis = 1)
y_val = val_data_df['future_won_game']

rf.fit(x, y)

preds = rf.predict(x_val)

accuracy_score(preds, y_val)

C:\Users\trist\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5099750623441397

In [50]:

df_interaction = pd.DataFrame()

df_interaction['future_won_game'] = all_data_df['future_won_game']

columns_list = training_data_df.columns.tolist()

for i in columns_list:
    df_interaction[i] =  all_data_df[i]
    for j in columns_list:
        if  i == 'future_won_game' or j == 'future_won_game':
            continue
        if columns_list.index(i) >=columns_list.index(j):
            continue
        df_interaction[f'{i}_mul_{j}'] = all_data_df[i]*all_data_df[j]
        
df_interaction.head()



,future_won_game,past_5_games_avg_won_game,past_5_games_avg_won_game_mul_past_5_games_avg_score,past_5_games_avg_won_game_mul_past_5_games_avg_scorePerRound,past_5_games_avg_won_game_mul_past_5_games_avg_killsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_kills,past_5_games_avg_won_game_mul_past_5_games_avg_deaths,past_5_games_avg_won_game_mul_past_5_games_avg_assists,past_5_games_avg_won_game_mul_past_5_games_avg_kdRatio,past_5_games_avg_won_game_mul_past_5_games_avg_damage,past_5_games_avg_won_game_mul_past_5_games_avg_damagePerRound,past_5_games_avg_won_game_mul_past_5_games_avg_singleKills,past_5_games_avg_won_game_mul_past_5_games_avg_doubleKills,past_5_games_avg_won_game_mul_past_5_games_avg_tripleKills,past_5_games_avg_won_game_mul_past_5_games_avg_quadraKills,past_5_games_avg_won_game_mul_past_5_games_avg_pentaKills,past_5_games_avg_won_game_mul_past_5_games_avg_multiKills,past_5_games_avg_won_game_mul_past_5_games_avg_grenadeCasts,past_5_games_avg_won_game_mul_past_5_games_avg_ability1Casts,past_5_games_avg_won_game_mul_past_5_games_avg_ability2Casts,past_5_games_avg_won_game_mul_past_5_games_avg_ultimateCasts,past_5_games_avg_won_game_mul_past_5_games_avg_grenadeCastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_ability1CastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_ability2CastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_ultimateCastsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_plants,past_5_games_avg_won_game_mul_past_5_games_avg_defuses,past_5_games_avg_won_game_mul_past_5_games_avg_firstKills,past_5_games_avg_won_game_mul_past_5_games_avg_firstDeaths,past_5_games_avg_won_game_mul_past_5_games_avg_esr,past_5_games_avg_won_game_mul_past_5_games_avg_firstKillsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_firstDeathsPerRound,past_5_games_avg_won_game_mul_past_5_games_avg_econRating,past_5_games_avg_won_game_mul_past_5_games_avg_hsAccuracy,past_5_games_avg_won_game_mul_past_5_games_avg_kast,past_5_games_avg_won_game_mul_past_5_games_avg_role_Controller,past_5_games_avg_won_game_mul_past_5_games_avg_role_Initiator,past_5_games_avg_won_game_mul_past_5_games_avg_role_Duelist,past_5_games_avg_won_game_mul_past_5_games_avg_role_Sentinel,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Astra,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Breach,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Brimstone,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Chamber,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Cypher,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Fade,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Jett,past_5_games_avg_won_game_mul_past_5_games_avg_agent_KAY/O,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Killjoy,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Neon,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Omen,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Phoenix,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Raze,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Reyna,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Sage,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Skye,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Sova,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Viper,past_5_games_avg_won_game_mul_past_5_games_avg_agent_Yoru,past_5_games_avg_score,past_5_games_avg_score_mul_past_5_games_avg_scorePerRound,past_5_games_avg_score_mul_past_5_games_avg_killsPerRound,past_5_games_avg_score_mul_past_5_games_avg_kills,past_5_games_avg_score_mul_past_5_games_avg_deaths,past_5_games_avg_score_mul_past_5_games_avg_assists,past_5_games_avg_score_mul_past_5_games_avg_kdRatio,past_5_games_avg_score_mul_past_5_games_avg_damage,past_5_games_avg_score_mul_past_5_games_avg_damagePerRound,past_5_games_avg_score_mul_past_5_games_avg_singleKills,past_5_games_avg_score_mul_past_5_games_avg_doubleKills,past_5_games_avg_score_mul_past_5_games_avg_tripleKills,past_5_ga

In [51]:
pd.set_option('max_colwidth', 400)



fi_list = list()

for i in df_interaction.columns:
    
    if i == 'future_won_game':
        continue
        
    rel = linregress(df_interaction['future_won_game'], df_interaction[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False]).head(20)

,column,slope,intercept,rvalue,r2,pvalue,stderr
352,past_5_games_avg_assists_mul_past_5_games_avg_econRating,18.959973,273.045776,0.079026,0.006245,0.000004,4.089704
402,past_5_games_avg_kdRatio_mul_past_5_games_avg_econRating,8.894615,68.346004,0.071721,0.005144,0.000027,2.115195
328,past_5_games_avg_assists_mul_past_5_games_avg_kdRatio,0.394158,5.163161,0.070905,0.005028,0.000033,0.094817
379,past_5_games_avg_kdRatio_mul_past_5_games_avg_damage,317.073029,3131.262568,0.070281,0.004939,0.000039,76.953909
453,past_5_games_avg_damage_mul_past_5_games_avg_kast,9819.313200,192830.783792,0.069638,0.004849,0.000046,2405.288757
330,past_5_games_avg_assists_mul_past_5_games_avg_damagePerRound,37.865313,641.841783,0.069421,0.004819,0.000048,9.304418
451,past_5_games_avg_damage_mul_past_5_games_avg_econRating,13162.965240,164622.776600,0.068991,0.004760,0.000054,3254.706810
380,past_5_games_avg_kdRatio_mul_past_5_games_avg_damagePerRound,16.576735,156.823935,0.068459,0.004687,0.000061,4.130781
378,past_5_games_avg_kdRatio,0.068766,1.084524,0.067898,0.004610,0.000070,0.017278
382,past_5_games_avg_kdRatio_mul_past_5_games_avg_doubleKills,0.311614,2.968822,0.067863,0.004605,0.000071,0.078337


In [52]:
fi_list = list()

for i in all_data_df.columns:
    if i == 'future_won_game':
        continue
    rel = linregress(all_data_df['future_won_game'], all_data_df[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False])

,column,slope,intercept,rvalue,r2,pvalue,stderr
7,past_5_games_avg_kdRatio,0.068766,1.084524,0.067898,4.610137e-03,0.000070,0.017278
8,past_5_games_avg_damage,107.674264,2687.315634,0.067162,4.510701e-03,0.000084,27.352380
9,past_5_games_avg_damagePerRound,4.735277,133.574203,0.066120,4.371805e-03,0.000109,1.221942
31,past_5_games_avg_econRating,2.308761,57.663218,0.064016,4.098042e-03,0.000179,0.615440
2,past_5_games_avg_scorePerRound,-42.469955,253.138138,-0.058555,3.428654e-03,0.000610,12.381154
3,past_5_games_avg_killsPerRound,-0.132837,0.856644,-0.057680,3.327026e-03,0.000736,0.039315
11,past_5_games_avg_doubleKills,0.109750,2.501194,0.054105,2.927362e-03,0.001544,0.034635
28,past_5_games_avg_esr,4.547449,60.503588,0.052328,2.738250e-03,0.002198,1.483962
6,past_5_games_avg_assists,0.146911,4.792837,0.044291,1.961693e-03,0.009563,0.056663
10,past_5_games_avg_singleKills,0.115767,5.820011,0.043904,1.927588e-03,0.010211,0.045045



## Agent pick prediction data set creation

In [53]:
all_records, all_agent_records = get_all_processed_data()


In [54]:
# agents = list()

# for i in all_records:
#     agents.append(i['agent'])
    
# sorted(list(set(agents)))

In [55]:
maps = list()

for i in all_records:
    maps.append(i['map_pick'])
    
sorted(list(set(maps)))

['Ascent', 'Bind', 'Breeze', 'Fracture', 'Haven', 'Icebox', 'Pearl', 'Split']

In [56]:
all_agent_records_df = pd.DataFrame.from_dict(all_agent_records)
all_agent_records_df.head()

,agent_Astra,agent_Breach,agent_Brimstone,agent_Chamber,agent_Cypher,agent_Fade,agent_Jett,agent_KAY/O,agent_Killjoy,agent_Neon,agent_Omen,agent_Phoenix,agent_Raze,agent_Reyna,agent_Sage,agent_Skye,agent_Sova,agent_Viper,agent_Yoru,map_Ascent,map_Bind,map_Breeze,map_Fracture,map_Haven,map_Icebox,map_Pearl,map_Split,role_Controller,role_Initiator,role_Duelist,role_Sentinel,role_Controller*role_Initiator,role_Controller*role_Duelist,role_Controller*role_Sentinel,role_Initiator*role_Duelist,role_Initiator*role_Sentinel,role_Duelist*role_Sentinel,game_win
0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,2,1,1,2,1,2,1,2,1
1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,2,1,1,2,1,2,1,2,0
2,0,0,0,1,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2,2,0,0,0,2,2,4,0
3,0,0,0,1,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,3,0,0,0,0,0,6,1
4,0,0,0,1,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,2,2,0,2,2,0,0,4,1


In [57]:
train_df, val_df = train_test_split(all_agent_records_df)

train_x = train_df.drop('game_win', axis = 1)
train_y = train_df['game_win']

val_x = val_df.drop('game_win', axis = 1)
val_y = val_df['game_win']
train_x.head()

,agent_Astra,agent_Breach,agent_Brimstone,agent_Chamber,agent_Cypher,agent_Fade,agent_Jett,agent_KAY/O,agent_Killjoy,agent_Neon,agent_Omen,agent_Phoenix,agent_Raze,agent_Reyna,agent_Sage,agent_Skye,agent_Sova,agent_Viper,agent_Yoru,map_Ascent,map_Bind,map_Breeze,map_Fracture,map_Haven,map_Icebox,map_Pearl,map_Split,role_Controller,role_Initiator,role_Duelist,role_Sentinel,role_Controller*role_Initiator,role_Controller*role_Duelist,role_Controller*role_Sentinel,role_Initiator*role_Duelist,role_Initiator*role_Sentinel,role_Duelist*role_Sentinel
3239,0,0,1,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,2,2,0,2,2,0,0,4
5096,0,0,0,1,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2,2,0,2,2,0,0,4
731,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,2,1,1,2,1,1,2,2,1
1981,0,0,0,1,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,3,0,0,0,0,0,6
1129,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,1,0,0,0,1,0,0,0,0,0,1,1,2,1,1,2,1,2,1,2


In [58]:
rf = RandomForestClassifier()

rf.fit(train_x, train_y)

preds = rf.predict(val_x)

accuracy_score(preds, val_y)

0.4799713876967096

In [59]:
fi_list = list()
for i, j in zip(train_x.columns, rf.feature_importances_):
    fi_list.append({'column':i, 'importance':j})
    
pd.DataFrame.from_dict(fi_list).sort_values(by = ['importance'])[-10:]

,column,importance
8,agent_Killjoy,0.032693
22,map_Fracture,0.033149
23,map_Haven,0.034272
14,agent_Sage,0.037746
18,agent_Yoru,0.038214
11,agent_Phoenix,0.038905
3,agent_Chamber,0.041468
13,agent_Reyna,0.047674
12,agent_Raze,0.047700
6,agent_Jett,0.048861


In [60]:
rf = LogisticRegression()

rf.fit(train_x, train_y)

preds = rf.predict(val_x)

accuracy_score(preds, val_y)

C:\Users\trist\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.4971387696709585

In [61]:
fi_list = list()

for i in train_x.columns:
    rel = linregress(train_y, train_x[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False])

,column,slope,intercept,rvalue,r2,pvalue,stderr
9,agent_Neon,-0.039140,0.221477,-0.048760,2.377526e-03,0.001589,0.012386
5,agent_Fade,0.034809,0.184564,0.043345,1.878831e-03,0.005002,0.012395
36,role_Duelist*role_Sentinel,0.094790,2.218600,0.036727,1.348858e-03,0.017407,0.039846
35,role_Initiator*role_Sentinel,0.057605,0.984180,0.032422,1.051167e-03,0.035809,0.027434
10,agent_Omen,-0.026740,0.365772,-0.027989,7.833584e-04,0.069994,0.014754
30,role_Sentinel,0.035762,1.284276,0.026786,7.174655e-04,0.082911,0.020618
11,agent_Phoenix,0.019771,0.218121,0.023560,5.550864e-04,0.127214,0.012960
16,agent_Sova,-0.017997,0.195110,-0.023123,5.346644e-04,0.134431,0.012021
7,agent_KAY/O,-0.017700,0.213806,-0.021925,4.807165e-04,0.155808,0.012469
0,agent_Astra,-0.007478,0.037392,-0.020738,4.300478e-04,0.179462,0.005569


In [62]:
train_df, val_df = train_test_split(all_agent_records_df)


df_interaction_train = pd.DataFrame()
df_interaction_val = pd.DataFrame()

df_interaction_train['game_win'] = train_df['game_win']
df_interaction_val['game_win'] = val_df['game_win']

columns_list = train_df.columns.tolist()

for i in columns_list:
    df_interaction_train[i] =  train_df[i]
    df_interaction_val[i] =  val_df[i]
    for j in columns_list:
        if  i == 'game_win' or j == 'game_win':
            continue
        if columns_list.index(i) >=columns_list.index(j):
            continue
        df_interaction_train[f'{i}_mul_{j}'] = train_df[i]*train_df[j]
        df_interaction_val[f'{i}_mul_{j}'] = val_df[i]*val_df[j]
        
df_interaction_train.head()

,game_win,agent_Astra,agent_Astra_mul_agent_Breach,agent_Astra_mul_agent_Brimstone,agent_Astra_mul_agent_Chamber,agent_Astra_mul_agent_Cypher,agent_Astra_mul_agent_Fade,agent_Astra_mul_agent_Jett,agent_Astra_mul_agent_KAY/O,agent_Astra_mul_agent_Killjoy,agent_Astra_mul_agent_Neon,agent_Astra_mul_agent_Omen,agent_Astra_mul_agent_Phoenix,agent_Astra_mul_agent_Raze,agent_Astra_mul_agent_Reyna,agent_Astra_mul_agent_Sage,agent_Astra_mul_agent_Skye,agent_Astra_mul_agent_Sova,agent_Astra_mul_agent_Viper,agent_Astra_mul_agent_Yoru,agent_Astra_mul_map_Ascent,agent_Astra_mul_map_Bind,agent_Astra_mul_map_Breeze,agent_Astra_mul_map_Fracture,agent_Astra_mul_map_Haven,agent_Astra_mul_map_Icebox,agent_Astra_mul_map_Pearl,agent_Astra_mul_map_Split,agent_Astra_mul_role_Controller,agent_Astra_mul_role_Initiator,agent_Astra_mul_role_Duelist,agent_Astra_mul_role_Sentinel,agent_Astra_mul_role_Controller*role_Initiator,agent_Astra_mul_role_Controller*role_Duelist,agent_Astra_mul_role_Controller*role_Sentinel,agent_Astra_mul_role_Initiator*role_Duelist,agent_Astra_mul_role_Initiator*role_Sentinel,agent_Astra_mul_role_Duelist*role_Sentinel,agent_Breach,agent_Breach_mul_agent_Brimstone,agent_Breach_mul_agent_Chamber,agent_Breach_mul_agent_Cypher,agent_Breach_mul_agent_Fade,agent_Breach_mul_agent_Jett,agent_Breach_mul_agent_KAY/O,agent_Breach_mul_agent_Killjoy,agent_Breach_mul_agent_Neon,agent_Breach_mul_agent_Omen,agent_Breach_mul_agent_Phoenix,agent_Breach_mul_agent_Raze,agent_Breach_mul_agent_Reyna,agent_Breach_mul_agent_Sage,agent_Breach_mul_agent_Skye,agent_Breach_mul_agent_Sova,agent_Breach_mul_agent_Viper,agent_Breach_mul_agent_Yoru,agent_Breach_mul_map_Ascent,agent_Breach_mul_map_Bind,agent_Breach_mul_map_Breeze,agent_Breach_mul_map_Fracture,agent_Breach_mul_map_Haven,agent_Breach_mul_map_Icebox,agent_Breach_mul_map_Pearl,agent_Breach_mul_map_Split,agent_Breach_mul_role_Controller,agent_Breach_mul_role_Initiator,agent_Breach_mul_role_Duelist,agent_Breach_mul_role_Sentinel,agent_Breach_mul_role_Controller*role_Initiator,agent_Breach_mul_role_Controller*role_Duelist,agent_Breach_mul_role_Controller*role_Sentinel,agent_Breach_mul_role_Initiator*role_Duelist,agent_Breach_mul_role_Initiator*role_Sentinel,agent_Breach_mul_role_Duelist*role_Sentinel,agent_Brimstone,agent_Brimstone_mul_agent_Chamber,agent_Brimstone_mul_agent_Cypher,agent_Brimstone_mul_agent_Fade,agent_Brimstone_mul_agent_Jett,agent_Brimstone_mul_agent_KAY/O,agent_Brimstone_mul_agent_Killjoy,agent_Brimstone_mul_agent_Neon,agent_Brimstone_mul_agent_Omen,agent_Brimstone_mul_agent_Phoenix,agent_Brimstone_mul_agent_Raze,agent_Brimstone_mul_agent_Reyna,agent_Brimstone_mul_agent_Sage,agent_Brimstone_mul_agent_Skye,agent_Brimstone_mul_agent_Sova,agent_Brimstone_mul_agent_Viper,agent_Brimstone_mul_agent_Yoru,agent_Brimstone_mul_map_Ascent,agent_Brimstone_mul_map_Bind,agent_Brimstone_mul_map_Breeze,agent_Brimstone_mul_map_Fracture,agent_Brimstone_mul_map_Haven,agent_Brimstone_mul_map_Icebox,agent_Brimstone_mul_map_Pearl,agent_Brimstone_mul_map_Split,agent_Brimstone_mul_role_Controller,agent_Brimstone_mul_role_Initiator,agent_Brimstone_mul_role_Duelist,agent_Brimstone_mul_role_Sentinel,agent_Brimstone_mul_role_Controller*role_Initiator,agent_Brimstone_mul_role_Controller*role_Duelist,agent_Brimstone_mul_role_Controller*role_Sentinel,agent_Brimstone_mul_role_Initiator*role_Duelist,agent_Brimstone_mul_role_Initiator*role_Sentinel,agent_Brimstone_mul_role_Duelist*role_Sentinel,agent_Chamber,agent_Chamber_mul_agent_Cypher,agent_Chamber_mul_agent_Fade,agent_Chamber_mul_agent_Jett,agent_Chamber_mul_agent_KAY/O,agent_Chamber_mul_agent_Killjoy,agent_Chamber_mul_agent_Neon,agent_Chamber_mul_agent_Omen,agent_Chamber_mul_agent_Phoenix,agent_Chamber_mul_agent_Raze,agent_Chamber_mul_agent_Reyna,agent_Chamber_mul_agent_Sage,agent_Chamber_mul_agent_Skye,agent_Chamber_mul_agent_Sova,agent_Chamber_mul_agent_Viper,agent_Chamber_mul_agent_Yoru,agent_Chamber_mul_map_Ascent,agent_Chamber_mul_map_Bind,agent_Chamber_mu

In [63]:
df_interaction_train.shape, df_interaction_val.shape

((4192, 704), (1398, 704))

In [64]:
rf = RandomForestClassifier(n_estimators=100, max_features=.5)

rf.fit( df_interaction_train.drop('game_win', axis = 1), df_interaction_train['game_win'])

preds = rf.predict(df_interaction_val.drop('game_win', axis = 1))

accuracy_score(preds, df_interaction_val['game_win'])

0.5092989985693849

In [65]:
rf = LogisticRegression()

rf.fit( df_interaction_train.drop('game_win', axis = 1), df_interaction_train['game_win'])

preds = rf.predict(df_interaction_val.drop('game_win', axis = 1))

accuracy_score(preds, df_interaction_val['game_win'])

C:\Users\trist\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5007153075822603

In [66]:

from sklearn.decomposition import PCA


rf = LogisticRegression()
pca = PCA(n_components=50)

x_pca = pca.fit_transform(df_interaction_train.drop('game_win', axis = 1))
x_val_pca = pca.transform(df_interaction_val.drop('game_win', axis = 1))

rf.fit(x_pca, df_interaction_train['game_win'])

preds = rf.predict(x_val_pca)

accuracy_score(preds, df_interaction_val['game_win'])

0.5207439198855508

In [67]:
rf = RandomForestClassifier(n_estimators=100, max_features=.5)
pca = PCA(n_components=50)

x_pca = pca.fit_transform(df_interaction_train.drop('game_win', axis = 1))
x_val_pca = pca.transform(df_interaction_val.drop('game_win', axis = 1))

rf.fit(x_pca, df_interaction_train['game_win'])

preds = rf.predict(x_val_pca)

accuracy_score(preds, df_interaction_val['game_win'])

0.5164520743919886

In [68]:

# for n_components in range(1, 100):
#     rf = SVC()
#     pca = PCA(n_components=n_components)

#     x_pca = pca.fit_transform(df_interaction_train.drop('game_win', axis = 1))
#     x_val_pca = pca.transform(df_interaction_val.drop('game_win', axis = 1))

#     rf.fit(x_pca, df_interaction_train['game_win'])

#     preds = rf.predict(x_val_pca)

#     print(n_components, accuracy_score(preds, df_interaction_val['game_win']))



In [69]:
fi_list = list()

for i in df_interaction_train.columns:
    if i == 'game_win':
        continue
    rel = linregress(df_interaction_train['game_win'], df_interaction_train[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False]).head(50)

,column,slope,intercept,rvalue,r2,pvalue,stderr
93,agent_Brimstone_mul_map_Fracture,0.017816,0.025226,0.049073,0.002408,0.001482,0.005602
317,agent_Neon_mul_role_Duelist,-0.089004,0.489291,-0.046501,0.002162,0.002600,0.029537
297,agent_Neon,-0.035306,0.217515,-0.044140,0.001948,0.004257,0.012345
206,agent_Fade_mul_role_Duelist*role_Sentinel,0.071321,0.313184,0.042948,0.001844,0.005417,0.025631
490,agent_Sova_mul_role_Initiator*role_Duelist,-0.076328,0.439791,-0.040246,0.001620,0.009160,0.029275
316,agent_Neon_mul_role_Initiator,-0.035021,0.177059,-0.040128,0.001610,0.009367,0.013472
319,agent_Neon_mul_role_Controller*role_Initiator,-0.030444,0.134222,-0.039529,0.001563,0.010479,0.011889
511,agent_Viper_mul_role_Initiator*role_Sentinel,0.044578,0.221323,0.038984,0.001520,0.011594,0.017652
184,agent_Fade_mul_agent_Sage,0.023889,0.095193,0.038624,0.001492,0.012387,0.009548
484,agent_Sova_mul_role_Initiator,-0.044600,0.274155,-0.038451,0.001478,0.012785,0.017906


In [70]:
from sklearn.decomposition import PCA

rf = LogisticRegression()
pca = PCA()

x_pca = pca.fit_transform(train_x)
x_val_pca = pca.transform(val_x)

rf.fit(x_pca, train_y)

preds = rf.predict(x_val_pca)

accuracy_score(preds, val_y)

0.49856938483547925

In [71]:

# from sklearn.decomposition import PCA

# rf = SVC()
# pca = PCA()

# x_pca = pca.fit_transform(train_x)
# x_val_pca = pca.transform(val_x)

# rf.fit(x_pca, train_y)

# preds = rf.predict(x_val_pca)

# accuracy_score(preds, val_y)

In [72]:
model= RandomForestClassifier(n_estimators=100, max_features = .9, max_samples=.5)

model.fit(train_x, train_y)

preds = model.predict(val_x)

accuracy_score(preds, val_y)

0.4771101573676681

In [73]:
# model.predict(val_x)

In [74]:
# model.predict_proba(val_x)[0][-1]

In [75]:
model= RandomForestClassifier(n_estimators=20, max_features = .5)

model.fit(train_x, train_y)

preds = model.predict(val_x)

accuracy_score(preds, val_y)

0.48569384835479257

In [76]:
model= RandomForestClassifier(n_estimators=20, max_features = .5)


train_df, val_df = train_test_split(all_agent_records_df)

all_x = all_agent_records_df.drop('game_win', axis = 1)
all_y = all_agent_records_df['game_win']

model.fit(all_x, all_y)


RandomForestClassifier(max_features=0.5, n_estimators=20)

In [77]:
def create_interactions(df: pd.DataFrame) -> pd.DataFrame:
    df_interaction = pd.DataFrame()

    df_interaction['game_win'] = df['game_win']

    columns_list = sorted(df.columns.tolist())

    for i in columns_list:
        df_interaction[i] = df[i]
        for j in columns_list:
            if  i == 'game_win' or j == 'game_win':
                continue
            if columns_list.index(i) >=columns_list.index(j):
                continue
            df_interaction[f'{i}_mul_{j}'] = df[i]*df[j]
    df_interaction.index = df.index
    return df_interaction

In [78]:
train_df, val_df = train_test_split(all_agent_records_df)


model = SVC(probability=True)
pca = PCA(n_components=12)

df_train = create_interactions(train_df)
df_val = create_interactions(val_df)

x_train_pca = pca.fit_transform(df_train.drop('game_win', axis = 1))
x_val_pca = pca.fit_transform(df_val.drop('game_win', axis = 1))

model.fit(x_train_pca, df_train['game_win'])

preds = model.predict(x_val_pca)
accuracy_score(preds, val_y)

0.4742489270386266

In [79]:
from sklearn.linear_model import LogisticRegression

train_df, val_df = train_test_split(all_agent_records_df)

model = LogisticRegression()
pca = PCA(n_components=12)

df_train = create_interactions(train_df)
df_val = create_interactions(val_df)

x_train_pca = pca.fit_transform(df_train.drop('game_win', axis = 1))
x_val_pca = pca.fit_transform(df_val.drop('game_win', axis = 1))

model.fit(x_train_pca, df_train['game_win'])

preds = model.predict(x_val_pca)
accuracy_score(preds, val_df['game_win'])
# preds

0.5092989985693849

In [80]:
train_df, val_df = train_test_split(all_agent_records_df)


In [ ]:
import random
results = list()

counter = 0

while True:
    counter += 1
    if counter > 1000:
        break
    i = random.randint(10, 100)
    j = random.randint(10, 99) * .01
    k = random.randint(1, 100)
    max_depth = random.randint(2, 10)
    

    model = RandomForestClassifier(n_estimators=i, max_features = j, max_depth=max_depth)
    pca = PCA(n_components=k)

    df_train_int = create_interactions(train_df)
    df_val_int = create_interactions(val_df)

    x_train_pca = pca.fit_transform(df_train_int.drop('game_win', axis = 1))
    x_val_pca = pca.fit_transform(df_val_int.drop('game_win', axis = 1))
    
    x_train_pca_df = pd.DataFrame(columns = [f'pca_{col}' for col in range(k)],
                                 data = x_train_pca, index = train_df.index)
    x_val_pca_df = pd.DataFrame(columns = [f'pca_{col}' for col in range(k)],
                             data = x_val_pca, index = val_df.index)
    
    df_concat_train = pd.concat([x_train_pca_df, train_df.drop('game_win', axis=1)], axis=1)
    df_concat_val = pd.concat([x_val_pca_df, val_df.drop('game_win', axis=1)], axis=1)

    model.fit(df_concat_train, train_df['game_win'])

    preds = model.predict(df_concat_val)
    score= accuracy_score(preds, val_df['game_win'])

    results.append({'n_estimators':i,
                   'max_features':j,
                   'n_components':k,
                    'max_depth':max_depth,
                   'score':score})
    if score >= sorted(results, key = lambda x: x['score'], reverse = True)[0]['score']:
        print(counter, i, j, k, score, sorted(results, key = lambda x: x['score'], reverse = True)[0])
        
            
sorted(results, key = lambda x: x['score'], reverse = True)[:10]

1 28 0.52 31 0.525035765379113 {'n_estimators': 28, 'max_features': 0.52, 'n_components': 31, 'max_depth': 4, 'score': 0.525035765379113}
6 21 0.91 56 0.5264663805436338 {'n_estimators': 21, 'max_features': 0.91, 'n_components': 56, 'max_depth': 4, 'score': 0.5264663805436338}
8 41 0.8200000000000001 12 0.5336194563662375 {'n_estimators': 41, 'max_features': 0.8200000000000001, 'n_components': 12, 'max_depth': 6, 'score': 0.5336194563662375}
11 91 0.84 65 0.5400572246065808 {'n_estimators': 91, 'max_features': 0.84, 'n_components': 65, 'max_depth': 7, 'score': 0.5400572246065808}
61 86 0.76 44 0.5586552217453505 {'n_estimators': 86, 'max_features': 0.76, 'n_components': 44, 'max_depth': 5, 'score': 0.5586552217453505}
104 99 0.9500000000000001 92 0.5600858369098712 {'n_estimators': 99, 'max_features': 0.9500000000000001, 'n_components': 92, 'max_depth': 10, 'score': 0.5600858369098712}


In [ ]:
x_train_pca_df.shape, train_df.shape , 

In [ ]:
# sorted(results, key = lambda x: x['score'], reverse = True)

In [ ]:
from sklearn.linear_model import LogisticRegression

train_df, val_df = train_test_split(all_agent_records_df)
n_components = 39
n_estimators = 92
max_features = 0.95
max_depth = 10

model = RandomForestClassifier(n_estimators=n_estimators, max_features = max_features, max_depth=max_depth)
pca = PCA(n_components=n_components)

df_train_int = create_interactions(train_df)
df_val_int = create_interactions(val_df)

x_train_pca = pca.fit_transform(df_train_int.drop('game_win', axis = 1))
x_val_pca = pca.fit_transform(df_val_int.drop('game_win', axis = 1))

x_train_pca_df = pd.DataFrame(columns = [f'pca_{col}' for col in range(n_components)],
                                 data = x_train_pca, index = train_df.index)
x_val_pca_df = pd.DataFrame(columns = [f'pca_{col}' for col in range(n_components)],
                         data = x_val_pca, index = val_df.index)

df_concat_train = pd.concat([x_train_pca_df, train_df.drop('game_win', axis=1)], axis=1)
df_concat_val = pd.concat([x_val_pca_df, val_df.drop('game_win', axis=1)], axis=1)



model.fit(df_concat_train, df_train['game_win'])

preds = model.predict(df_concat_val)
accuracy_score(preds, val_df['game_win'])


In [ ]:
fi_list = list()
for i, j in zip(df_concat_train.columns, model.feature_importances_):
    fi_list.append({'column':i, 'importance':j})
    
pd.DataFrame.from_dict(fi_list).sort_values(by = ['importance'])[-10:]

In [ ]:
fi_list = list()


fi_df = pd.concat([x_train_pca_df, train_df], axis=1)

for i in fi_df.columns:
    if i == 'game_win':
        continue
    rel = linregress(fi_df['game_win'], fi_df[i])
    fi_list.append({'column': i,
                   'slope':rel.slope,
                   'intercept':rel.intercept,
                   'rvalue':rel.rvalue,
                    'r2':rel.rvalue*rel.rvalue,
                   'pvalue':rel.pvalue,
                   'stderr':rel.stderr})

pd.DataFrame.from_dict(fi_list).sort_values(by = ['r2'], ascending = [False]).head(50)

In [ ]:

model = RandomForestClassifier(n_estimators=n_estimators, max_features = max_features, max_depth=max_depth)
pca = PCA(n_components=n_components)

df_all = create_interactions(all_agent_records_df)

x_pca = pca.fit_transform(df_all.drop('game_win', axis = 1))
x_pca_df = pd.DataFrame(columns = [f'pca_{col}' for col in range(n_components)],
                                 data = x_pca, index = df_all.index)
df_concat = pd.concat([x_pca_df, all_agent_records_df.drop('game_win', axis=1)], axis=1)

model.fit(df_concat, all_agent_records_df['game_win'])

preds = model.predict(df_concat)
preds.shape, df_concat.shape, df_all.shape

In [ ]:
def predict_best_lineup(model, vectorizer, map_pick, current_agent_list):
    
    agent_score = list()
    
    agent_tuples = list()
    
    for i1 in all_agents_list:
        for i2 in all_agents_list:
            for i3 in all_agents_list:
                for i4 in all_agents_list:
                    for i5 in all_agents_list:
                        if len(set([i1, i2, i3, i4, i5])) < 5:
                            continue
                        else:
                            agent_tuples.append(tuple(sorted([i1, i2, i3, i4, i5])))
    agent_tuples = list(set(agent_tuples))
    matched_agent_tuples = list()
    
    for i in agent_tuples:
        match = True
        for j in current_agent_list:
            if j not in i:
                match = False
        if match:
            matched_agent_tuples.append(i)
    
    inputs = list()
    features = list()
    
    for i in matched_agent_tuples:
        inputs.append({'agent1_name': i[0], 'agent2_name': i[1], 'agent3_name': i[2], 'agent4_name': i[3], 'agent5_name': i[4]})
        features.append(create_agent_row(list(i), map_pick, None))
    
    features_df = pd.DataFrame.from_dict(features)
    features_df_interactions = create_interactions(features_df)
    features_df_interactions_pca = pca.fit_transform(features_df_interactions.drop('game_win', axis = 1))
    features_df_interactions_pca_df = pd.DataFrame(columns = [f'pca_{col}' for col in range(n_components)],
                                 data = features_df_interactions_pca, index = features_df.index)
    df_concat = pd.concat([features_df_interactions_pca_df, features_df.drop('game_win', axis=1)], axis=1)

    inputs_df = pd.DataFrame.from_dict(inputs)
    
    inputs_df.index = features_df.index

    inputs_df['win_prob'] = model.predict_proba(df_concat)[:,-1]
    return inputs_df.sort_values('win_prob', ascending = False)


map_pick = 'Icebox'
current_agent_list = ['Phoenix',
                     'Reyna',
                      'Raze',
                     ]

predict_best_lineup(model, pca, map_pick, current_agent_list)

In [ ]:
['Ascent', 'Bind', 'Breeze', 'Fracture', 'Haven', 'Icebox', 'Pearl', 'Split']

In [ ]:
map_pick = 'Ascent'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Bind'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Breeze'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Fracture'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Haven'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Icebox'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Pearl'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()

In [ ]:
map_pick = 'Split'
current_agent_list = []

predict_best_lineup(model,pca, map_pick, current_agent_list).head()

In [ ]:
predict_best_lineup(model,pca, map_pick, current_agent_list).tail()